In [1]:
import numpy as np
import sys, platform
import numpy as np
from qiskit.circuit.library import iqp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator

print("Python full:", sys.version)          # e.g. '3.12.5 (main, ...)'
print("Python tuple:", sys.version_info)     # e.g. sys.version_info(major=3, minor=12, micro=5, ...)
print("Python short:", platform.python_version())  # e.g. '3.12.5'
print("Executable:", sys.executable)         # path to the kernel's python


Python full: 3.12.9 | packaged by conda-forge | (main, Mar  4 2025, 22:44:42) [Clang 18.1.8 ]
Python tuple: sys.version_info(major=3, minor=12, micro=9, releaselevel='final', serial=0)
Python short: 3.12.9
Executable: /opt/homebrew/anaconda3/envs/qiskit2x/bin/python


In [2]:
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
from qiskit.circuit.library import efficient_su2

service = QiskitRuntimeService()
print('the backends available are: ', service.backends())
backend = service.backends()[0]

n_qubits=16
# circuit = efficient_su2(127, entanglement='linear')
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

param_values = np.random.rand(circuit.num_parameters)
sampler = SamplerV2(mode=backend)
job = sampler.run([(isa_circuit, param_values)])
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")

the backends available are:  [<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
>>> Job ID: d3k0fj9fk6qs73em1jvg
>>> Job Status: QUEUED


In [3]:
print(job.result())

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4096, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-09 19:06:55', stop='2025-10-09 19:06:57', size=4096>)])}, 'version': 2})


### Try out Sampler options - IBM Runtime Provider

In [13]:
from qiskit.quantum_info import SparsePauliOp, random_hermitian
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

service = QiskitRuntimeService()
print('the backends available are: ', service.backends())
backend = service.backends()[0]

n_qubits=16
mat = np.real(random_hermitian(n_qubits, seed=1234))
circuit = iqp(mat)
circuit.measure_all()

pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(circuit)

param_values = np.random.rand(circuit.num_parameters)
sampler = SamplerV2(mode=backend)

### set sampler options
sampler.options.default_shots = 2048

job = sampler.run([(isa_circuit, param_values)],shots=4)  # can override defaults when you run it
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
print('results are', job.result())

the backends available are:  [<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]
>>> Job ID: d3k1ve1fk6qs73em34o0
>>> Job Status: QUEUED
results are PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4, num_bits=16>)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-10-09 20:48:58', stop='2025-10-09 20:48:59', size=4>)])}, 'version': 2})


### Twirling

In [16]:
sampler.options.twirling.enable_gates = True
sampler.options.twirling.enable_measure = True
sampler.options.twirling.strategy = "active-accum"      # good default
sampler.options.twirling.num_randomizations = 20        # e.g., 8–32
sampler.options.twirling.shots_per_randomization = "auto"  # splits your shots

job = sampler.run([(isa_circuit, param_values)],shots=4)  # can override defaults when you run it
print(f">>> Job ID: {job.job_id()}")
print(f">>> Job Status: {job.status()}")
print('results are', job.result())

>>> Job ID: d3k2509fk6qs73em3b30
>>> Job Status: QUEUED
results are PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(), num_shots=4, num_bits=16>)), metadata={'circuit_metadata': {}, 'num_randomizations': 20})], metadata={'execution': {'execution_spans': ExecutionSpans([TwirledSliceSpanV2(<start='2025-10-09 21:00:52', stop='2025-10-09 21:00:52', size=20>)])}, 'version': 2})


In [17]:
## Error mitigation